# Opción GPT

In [68]:
import random
import math

def load_cities_from_file(file_path):
    """Load city coordinates from a text file."""
    cities = []
    with open(file_path, 'r') as file:
        lines = file.readlines()[1:]  # Skip the first line (header)
        for line in lines:
            parts = line.split()
            if len(parts) == 3:
                x, y = float(parts[1]), float(parts[2])
                cities.append((x, y))
    return cities

In [ ]:
def calculate_distance(city1, city2):
    """Calculate the Euclidean distance between two cities."""
    return round(math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2))

def nearest_neighbor(cities):
    """Generate an initial route using the Nearest Neighbor heuristic."""
    unvisited = set(range(len(cities)))
    route = []
    current_city = random.choice(list(unvisited))  # Start from a random city
    route.append(current_city)
    unvisited.remove(current_city)

    while unvisited:
        next_city = min(unvisited, key=lambda city: calculate_distance(cities[current_city], cities[city]))
        route.append(next_city)
        unvisited.remove(next_city)
        current_city = next_city

    return route, calculate_route_distance(route, cities)

def calculate_route_distance(route, cities):
    """Calculate the total distance of a given route."""
    distance = 0
    for i in range(len(route)):
        distance += calculate_distance(cities[route[i]], cities[route[(i + 1) % len(route)]])
    return distance

def tabu_search(route, cities, max_iterations=500, tabu_size=50):
    """Improve the route using Tabu Search."""
    best_route = route[:]
    best_distance = calculate_route_distance(route, cities)
    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(best_route)
        best_candidate = None
        best_candidate_distance = float('inf')

        for neighbor in neighbors:
            if neighbor not in tabu_list:
                distance = calculate_route_distance(neighbor, cities)
                if distance < best_candidate_distance:
                    best_candidate = neighbor
                    best_candidate_distance = distance

        if best_candidate_distance < best_distance:
            best_route = best_candidate[:]
            best_distance = best_candidate_distance

        tabu_list.append(best_candidate)
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

    return best_route, best_distance

def generate_neighbors(route):
    """Generate neighbors by swapping two cities in the route."""
    neighbors = []
    for i in range(len(route)):
        for j in range(i + 1, len(route)):
            neighbor = route[:]
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            neighbors.append(neighbor)
    return neighbors

def genetic_algorithm(cities, initial_route=None, population_size=250, generations=100, mutation_rate=0.5):
    """Optimize the route using a Genetic Algorithm."""
    def crossover(parent1, parent2):
        """Perform order crossover between two parents."""
        start, end = sorted(random.sample(range(len(parent1)), 2))
        child = [None] * len(parent1)
        child[start:end] = parent1[start:end]
        pos = end
        for gene in parent2:
            if gene not in child:
                if pos >= len(child):
                    pos = 0
                child[pos] = gene
                pos += 1
        return child

    def mutate(route):
        """Mutate a route by swapping two cities."""
        if random.random() < mutation_rate:
            i, j = random.sample(range(len(route)), 2)
            route[i], route[j] = route[j], route[i]

    # Initialize population, including the initial route if provided
    population = [random.sample(range(len(cities)), len(cities)) for _ in range(population_size - 1)]
    if initial_route:
        population.append(initial_route)

    for _ in range(generations):
        population = sorted(population, key=lambda route: calculate_route_distance(route, cities))
        next_generation = population[:population_size // 2]  # Select top half

        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(next_generation, 2)
            child = crossover(parent1, parent2)
            mutate(child)
            next_generation.append(child)

        population = next_generation

    best_route = min(population, key=lambda route: calculate_route_distance(route, cities))
    return best_route, calculate_route_distance(best_route, cities)

# Example usage with Qatar data:
file_path = "Qatar.txt"
cities = load_cities_from_file(file_path)

# Step 1: Nearest Neighbor
nn_route, nn_distance = nearest_neighbor(cities)
print(f"Nearest Neighbor: Route: {nn_route}, Distance: {nn_distance}")

# Step 2: Tabu Search
optimized_route, optimized_distance = tabu_search(nn_route, cities)
print(f"Tabu Search: Route: {optimized_route}, Distance: {optimized_distance}")

# Step 3: Genetic Algorithm with Tabu Search Output
ga_route, ga_distance = genetic_algorithm(cities, initial_route=optimized_route)
print(f"Genetic Algorithm: Route: {ga_route}, Distance: {ga_distance}")


# Genético Memo

In [69]:
def tspGenetico(cities, n=250, rate=0.5, mutation_rate=0.2, generations=100, max_iter=1000):
    def distanciaTotal(orden, cities):
        total = 0.0
        num_points = len(orden)
        for i in range(num_points):
            if i == num_points - 1:
                total += calculate_distance(cities[orden[i]], cities[orden[0]])
            else:
                total += calculate_distance(cities[orden[i]], cities[orden[i + 1]])
        return total

    def calculate_distance(city1, city2):
        """Calculate the Euclidean distance between two cities."""
        return round(math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2))

    def nearestNeighborTsp(cities):
        num_points = len(cities)
        unvisited = set(range(1, num_points))
        current_point = 0
        path = [current_point]

        while unvisited:
            nearest_point = min(unvisited, key=lambda x: calculate_distance(cities[current_point], cities[x]))
            path.append(nearest_point)
            unvisited.remove(nearest_point)
            current_point = nearest_point

        path.append(0)  # Close the loop
        return path

    def swap(individual):
        start, end = random.sample(range(len(individual) - 1), 2)
        individual[start], individual[end] = individual[end], individual[start]
        return individual

    def mutate(individual):
        return swap(individual.copy())

    def crossover(parent1, parent2):
        start, end = sorted(random.sample(range(len(parent1) - 1), 2))
        child = [-1] * len(parent1)
        child[start:end] = parent1[start:end]
        fill_index = end
        for gene in parent2:
            if gene not in child:
                if fill_index == len(parent1) - 1:
                    fill_index = 0
                child[fill_index] = gene
                fill_index += 1
        child[-1] = child[0]  # Close the loop
        return child

    def generate_initial_population(initial_solution, population_size):
        population = [initial_solution]
        while len(population) < population_size:
            tour = list(range(len(cities)))
            random.shuffle(tour)
            tour.append(tour[0])  # Close the loop
            population.append(tour)
        return population

    def select_best(population, cities, num_best):
        distances = [distanciaTotal(ind, cities) for ind in population]
        selected = sorted(zip(population, distances), key=lambda x: x[1])[:num_best]
        return [x[0] for x in selected]

    def evolve_population(population, num_best, rate, mutation_rate):
        new_population = select_best(population, cities, num_best)
        while len(new_population) < len(population):
            if random.random() < mutation_rate:
                mutant = mutate(random.choice(new_population))
                new_population.append(mutant)
            else:
                parent1, parent2 = random.sample(new_population, 2)
                child = crossover(parent1, parent2)
                new_population.append(child)
        return new_population

    # Initial population
    initial_solution = nearestNeighborTsp(cities)
    population = generate_initial_population(initial_solution, n)

    best_distance = float('inf')
    best_solution = None

    for _ in range(generations):
        population = evolve_population(population, int(n * rate), rate, mutation_rate)
        current_best = min(population, key=lambda x: distanciaTotal(x, cities))
        current_best_distance = distanciaTotal(current_best, cities)
        if current_best_distance < best_distance:
            best_distance = current_best_distance
            best_solution = current_best

    return best_solution, best_distance

In [70]:
file_path = "Qatar.txt"
cities = load_cities_from_file(file_path)
tspGenetico(cities)[1]

11630.0

# Tabu Memo

In [72]:
import random
import heapq
import numpy as np
import math

def calculate_distance(city1, city2):
    """Calcula la distancia euclidiana entre dos ciudades."""
    return round(math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2))

def generate_distance_matrix(cities):
    """Genera una matriz de distancias a partir de las coordenadas de las ciudades."""
    n = len(cities)
    distance_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                distance_matrix[i][j] = calculate_distance(cities[i], cities[j])
    return distance_matrix

def dynamic_tabu_tenure(iteration, min_duration, max_duration, penalty_factor):
    base_tenure = min_duration + (max_duration - min_duration) * np.sin(np.pi * iteration / 100)
    return int(base_tenure * penalty_factor)

def calculate_tour_length(tour, distance_matrix):
    return sum(distance_matrix[tour[i]][tour[i+1]] for i in range(len(tour) - 1)) + distance_matrix[tour[0]][tour[-1]]

def intensify_search(tour, best_tour, best_length, iteration, intensify_factor, distance_matrix):
    if iteration % intensify_factor == 0:
        current_length = calculate_tour_length(tour, distance_matrix)
        if current_length > best_length:
            return best_tour[:], best_length
    return tour, calculate_tour_length(tour, distance_matrix)

def diversify_search(tour, n, diversify_factor, iteration):
    if iteration % diversify_factor == 0:
        segment_size = n // 10
        start = random.randint(0, n - segment_size)
        tour[start:start + segment_size] = random.sample(tour[start:start + segment_size], segment_size)
    return tour

def calculate_delta_penalty(i, j, tour, d, count, F):
    n = len(tour)
    delta = d[tour[i]][tour[j]] + d[tour[i+1]][tour[(j+1) % n]] - d[tour[i]][tour[i+1]] - d[tour[j]][tour[(j+1) % n]]
    penalty = F * (count[tour[i]][tour[j]] + count[tour[i+1]][tour[(j+1) % n]])
    return delta, penalty

def apply_tabu(tour, ir, jr, tabu, tabu_tenure, count, iteration):
    tabu[tour[ir]][tour[jr]] = tabu_tenure + iteration
    count[tour[ir]][tour[jr]] += 1

def tsp_TS(cities, tour, iterations, min_tabu_duration, max_tabu_duration, F):
    d = generate_distance_matrix(cities)
    n = len(tour)
    tabu = [[0] * n for _ in range(n)]
    count = [[0] * n for _ in range(n)]
    best_tour = tour[:]
    best_length = calculate_tour_length(tour, d)
    intensify_factor = 100
    diversify_factor = 500

    for iteration in range(iterations):
        tour = diversify_search(tour, n, diversify_factor, iteration)
        tour, length = intensify_search(tour, best_tour, best_length, iteration, intensify_factor, d)

        delta_penalty = float('inf')
        ir = jr = -1

        for i in range(n - 2):
            for j in range(i + 2, n if i > 0 else n - 1):
                delta, penalty = calculate_delta_penalty(i, j, tour, d, count, F)
                if delta + penalty < delta_penalty and (tabu[tour[i]][tour[j]] <= iteration or best_length + delta < best_length):
                    delta_penalty = delta + penalty
                    ir, jr = i, j

        if ir >= 0 and jr >= 0:
            tabu_tenure = dynamic_tabu_tenure(iteration, min_tabu_duration, max_tabu_duration, F)
            apply_tabu(tour, ir, jr, tabu, tabu_tenure, count, iteration)

            length += delta_penalty
            tour[ir+1:jr+1] = reversed(tour[ir+1:jr+1])
            if length < best_length:
                best_length = length
                best_tour = tour[:]

    best_tour.append(tour[0])
    return best_tour, best_length

def insertion_heuristic(cities):
    d = generate_distance_matrix(cities)
    n = len(d)
    tour = [0, np.argmin(d[0][1:]) + 1]
    remaining_cities = set(range(1, n)) - set(tour)
    total_distance = d[0][tour[1]]

    heap = []
    for city in remaining_cities:
        distance_to_tour = min((d[city][tour_node] + d[tour_node][city], city, tour_node) for tour_node in tour)
        heapq.heappush(heap, distance_to_tour)

    while remaining_cities:
        _, city, _ = heapq.heappop(heap)
        if city not in remaining_cities:
            continue

        min_increase = float('inf')
        best_position = 0
        for i in range(1, len(tour)):
            increase = (d[tour[i - 1]][city] +
                        d[city][tour[i]] -
                        d[tour[i - 1]][tour[i]])
            if increase < min_increase:
                min_increase = increase
                best_position = i

        tour.insert(best_position, city)
        total_distance += min_increase
        remaining_cities.remove(city)

        for remaining_city in remaining_cities:
            distance_to_tour = min((d[remaining_city][tour_node] + d[tour_node][remaining_city], remaining_city, tour_node) for tour_node in tour)
            heapq.heappush(heap, distance_to_tour)

    total_distance += d[tour[-1]][tour[0]]

    return tour, total_distance

def insert_taboo(cities, iterations=500, min_tabu_duration=5, max_tabu_duration=50, penalty_factor=2, verbose=False):
    initial_tour, initial_length = insertion_heuristic(cities)
    best_tour, best_length = tsp_TS(cities, initial_tour, iterations, min_tabu_duration, max_tabu_duration, penalty_factor)
    best_tour = [i+1 for i in best_tour]
    if verbose:
        print("Best tour:", best_tour, "\nBest length:", best_length)
    return best_tour, best_length

In [75]:
file_path = "Qatar.txt"
cities = load_cities_from_file(file_path)
insert_taboo(cities)[1]

10178

# Integrado

In [1]:
import random
import math

def load_cities_from_file(file_path):
    """Load city coordinates from a text file."""
    cities = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.split()
            if len(parts) == 3:
                x, y = float(parts[1]), float(parts[2])
                cities.append((x, y))
    return cities

In [25]:
import random
import math
import numpy as np

class FastGeneticTabuTSP:
    def __init__(self, cities, population_size=100, generations=50, 
                 mutation_rate=0.2, elite_rate=0.15):
        self.cities = cities
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.elite_rate = elite_rate
        self.distance_matrix = self._generate_distance_matrix()
        
    def _generate_distance_matrix(self):
        n = len(self.cities)
        matrix = [[0] * n for _ in range(n)]
        for i in range(n):
            for j in range(n):
                if i != j:
                    matrix[i][j] = self._calculate_distance(self.cities[i], self.cities[j])
        return matrix
    
    def _calculate_distance(self, city1, city2):
        return round(math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2))
    
    def _greedy_insertion_initial_population(self):
        def greedy_tour():
            n = len(self.cities)
            unvisited = set(range(1, n))
            tour = [0]
            
            while unvisited:
                last = tour[-1]
                next_city = min(unvisited, key=lambda x: self.distance_matrix[last][x])
                tour.append(next_city)
                unvisited.remove(next_city)
            
            return tour
        
        population = []
        for _ in range(self.population_size):
            tour = greedy_tour()
            # Añadir algo de aleatoriedad
            for _ in range(5):
                i, j = random.sample(range(len(tour)), 2)
                tour[i], tour[j] = tour[j], tour[i]
            population.append(tour)
        
        return population
    
    def _calculate_tour_length(self, tour):
        total = sum(self.distance_matrix[tour[i]][tour[i+1]] for i in range(len(tour) - 1))
        total += self.distance_matrix[tour[-1]][tour[0]]
        return total
    
    def _quick_two_opt(self, tour):
        """Versión rápida y segura de 2-opt"""
        n = len(tour)
        improved = True
        while improved:
            improved = False
            for i in range(1, n - 1):
                for j in range(i + 1, n):
                    # Calcular cambio en distancia con seguridad
                    old_distance = (
                        self.distance_matrix[tour[i-1]][tour[i]] + 
                        self.distance_matrix[tour[j]][tour[(j+1)%n]]
                    )
                    new_distance = (
                        self.distance_matrix[tour[i-1]][tour[j]] + 
                        self.distance_matrix[tour[i]][tour[(j+1)%n]]
                    )
                    
                    if new_distance < old_distance:
                        # Invertir segmento con seguridad
                        tour[i:j+1] = tour[i:j+1][::-1]
                        improved = True
                        break
                if improved:
                    break
        return tour
    
    def _partially_mapped_crossover(self, parent1, parent2):
        """Crossover de mapeo parcial"""
        n = len(parent1)
        # Seleccionar segmento
        start, end = sorted(random.sample(range(n), 2))
        
        # Crear hijo
        child = [None] * n
        child[start:end] = parent1[start:end]
        
        # Mapear el resto
        for i in range(n):
            if child[i] is None:
                current = parent2[i]
                while current in child:
                    current = parent2[child.index(current)]
                child[i] = current
        
        return child
    
    def solve(self, verbose=False):
        # Generar población inicial con inserción voraz
        population = self._greedy_insertion_initial_population()
        
        best_tour = min(population, key=self._calculate_tour_length)
        best_length = self._calculate_tour_length(best_tour)
        
        for generation in range(self.generations):
            # Ordenar población por fitness
            population_with_fitness = [(tour, self._calculate_tour_length(tour)) for tour in population]
            population_with_fitness.sort(key=lambda x: x[1])
            
            # Selección de élite
            elite_count = int(self.population_size * self.elite_rate)
            new_population = [tour for tour, _ in population_with_fitness[:elite_count]]
            
            while len(new_population) < self.population_size:
                # Selección por torneo
                parent1 = min(random.sample(population, 3), key=self._calculate_tour_length)
                parent2 = min(random.sample(population, 3), key=self._calculate_tour_length)
                
                # Crossover
                child = self._partially_mapped_crossover(parent1, parent2)
                
                # Mutación
                if random.random() < self.mutation_rate:
                    child = self._quick_two_opt(child)
                
                new_population.append(child)
            
            population = new_population
            
            # Actualizar mejor solución
            current_best = min(population, key=self._calculate_tour_length)
            current_best_length = self._calculate_tour_length(current_best)
            
            if current_best_length < best_length:
                best_tour = current_best
                best_length = current_best_length
                
                if verbose:
                    print(f"Generación {generation}: Mejor distancia = {best_length}")
        
        best_tour.append(best_tour[0])  # Cerrar el ciclo
        return best_tour, best_length

In [26]:
import plotly.graph_objects as go

def plot_route(cities, route, title="TSP Route"):
    """
    Graficar la ruta del TSP utilizando Plotly y rotando la gráfica 90º.

    :param cities: Lista de coordenadas de las ciudades [(x1, y1), (x2, y2), ..., (xn, yn)].
    :param route: Lista de índices que indica el orden de las ciudades visitadas en la ruta.
    :param title: Título de la gráfica.
    """
    # Cerrar todas las gráficas previas antes de mostrar una nueva
    # Plotly no requiere cerrar gráficos previos, cada figura se muestra por separado

    # Obtener las coordenadas de las ciudades según el orden de la ruta
    ordered_cities = [cities[i] for i in route]
    
    # Añadir la ciudad de inicio al final para formar un ciclo
    ordered_cities.append(ordered_cities[0])
    
    # Desempaquetar las coordenadas
    x, y = zip(*ordered_cities)

    # Crear la gráfica con Plotly, girando 90º (intercambiando x e y)
    fig = go.Figure()

    # Añadir la ruta
    fig.add_trace(go.Scatter(x=y, y=x, mode='lines+markers', marker=dict(size=6, color='blue'), name='Ruta'))

    # Añadir etiquetas de las ciudades
    for i, city in enumerate(ordered_cities[:-1]):  # No etiquetar el último (es el mismo que el primero)
        fig.add_trace(go.Scatter(x=[city[1]], y=[city[0]], mode='text', text=[str(i)], textposition="top center"))

    # Configurar el título y los ejes
    fig.update_layout(
        title=title,
        xaxis_title='Coordenada Y',  # Y ahora es lo que estaba en X
        yaxis_title='Coordenada X',  # X ahora es lo que estaba en Y
        showlegend=False,
        height=700,
        width=800
    )

    # Mostrar la gráfica
    fig.show()

In [27]:
file_path = "Qatar.txt"
cities = load_cities_from_file(file_path)
genetic_solver = FastGeneticTabuTSP(cities)
best_tour, best_length = genetic_solver.solve(verbose=True)
print("Best tour:", best_tour, "\nBest length:", best_length)

Generación 0: Mejor distancia = 9953
Generación 1: Mejor distancia = 9885
Generación 2: Mejor distancia = 9808
Generación 5: Mejor distancia = 9739
Generación 8: Mejor distancia = 9512
Generación 17: Mejor distancia = 9490
Generación 18: Mejor distancia = 9481
Generación 19: Mejor distancia = 9464
Generación 21: Mejor distancia = 9457
Generación 28: Mejor distancia = 9448
Generación 29: Mejor distancia = 9430
Generación 31: Mejor distancia = 9427
Best tour: [0, 5, 7, 15, 12, 22, 24, 13, 10, 6, 16, 25, 23, 20, 17, 32, 27, 28, 21, 26, 36, 44, 56, 59, 68, 73, 71, 77, 74, 75, 86, 79, 70, 81, 61, 58, 35, 62, 19, 64, 84, 85, 97, 89, 88, 93, 98, 100, 103, 110, 129, 126, 124, 125, 131, 133, 136, 139, 144, 155, 148, 145, 141, 137, 138, 153, 156, 152, 149, 143, 140, 151, 146, 158, 157, 161, 166, 164, 167, 176, 180, 183, 174, 172, 173, 178, 171, 163, 162, 160, 168, 175, 181, 193, 185, 182, 186, 189, 191, 190, 188, 187, 192, 184, 179, 177, 169, 170, 165, 159, 154, 150, 147, 142, 135, 130, 128, 134

In [28]:
plot_route(cities,best_tour)

In [ ]:
file_path = "Uruguay.txt"
cities = load_cities_from_file(file_path)
genetic_solver = FastGeneticTabuTSP(cities)
best_tour, best_length = genetic_solver.solve(verbose=True)
print("Best tour:", best_tour, "\nBest length:", best_length)